In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import os

# Check for GPU/TPU availability
if not tf.config.list_physical_devices('GPU'):
    print("No GPU detected. Training may be slower.")
else:
    print("GPU is available!")

# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to the range [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Optional: Use a smaller subset for quick testing (uncomment if needed)
# x_train = x_train[:10000]
# y_train = y_train[:10000]
# x_test = x_test[:2000]
# y_test = y_test[:2000]

# Load the pre-trained MobileNetV2 model without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

# Freeze the base model layers to retain pre-trained features
base_model.trainable = False

# Add new layers for CIFAR-10 classification
model = models.Sequential([
    layers.Resizing(96, 96),  # Resize CIFAR-10 images to 96x96
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(10, activation='softmax')  # 10 classes for CIFAR-10
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model's architecture
model.summary()

# Train only the newly added layers
print("\nTraining the new layers...")
history = model.fit(x_train, y_train, epochs=3, batch_size=128,  # Reduced epochs for faster debugging
                    validation_data=(x_test, y_test))

# Unfreeze the base model layers and fine-tune
print("\nFine-tuning the entire model...")
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(x_train, y_train, epochs=3, batch_size=128,  # Reduced epochs
                         validation_data=(x_test, y_test))

# Evaluate the model
print("\nEvaluating the model on the test set...")
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')


No GPU detected. Training may be slower.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resizing (Resizing)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_96 (Functional)     │ (None, 3, 3, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ ?                           │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)


Training the new layers...
Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 262s 653ms/step - accuracy: 0.6210 - loss: 1.1405 - val_accuracy: 0.7847 - val_loss: 0.6160
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 250s 640ms/step - accuracy: 0.7608 - loss: 0.6839 - val_accuracy: 0.7997 - val_loss: 0.5807
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 243s 622ms/step - accuracy: 0.7853 - loss: 0.6177 - val_accuracy: 0.8069 - val_loss: 0.5550

Fine-tuning the entire model...
Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 1313s 3s/step - accuracy: 0.6645 - loss: 1.1624 - val_accuracy: 0.7858 - val_loss: 0.6705
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 1495s 4s/step - accuracy: 0.8355 - loss: 0.5096 - val_accuracy: 0.8157 - val_loss: 0.5646
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 1434s 4s/step - accuracy: 0.8679 - loss: 0.4018 - val_accuracy: 0.8302 - val_loss: 0.5122

Evaluating the model on the test set...
313/313 - 51s - 162ms/step - accuracy: 0.8302 - loss: 0.5122

Test accuracy: 0.8302000164985657


In [7]:
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
import numpy as np

# Download the IMDB dataset using TensorFlow Datasets
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load IMDB dataset from TensorFlow datasets (train and test)
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

# Split the dataset into train and test
train_data, test_data = dataset['train'], dataset['test']

# Preprocess the data: remove stopwords, tokenize, and convert to string format
def preprocess_text(text):
    # Convert text to string and remove stopwords
    text_str = text.numpy().decode('utf-8')
    words = text_str.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

# Convert data to list and preprocess
train_texts = [preprocess_text(text) for text, label in train_data]
test_texts = [preprocess_text(text) for text, label in test_data]

train_labels = [label.numpy() for text, label in train_data]
test_labels = [label.numpy() for text, label in test_data]

# Convert to TF-IDF features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_texts)
X_test_tfidf = vectorizer.transform(test_texts)

# Train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, train_labels)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate model
print("Accuracy:", accuracy_score(test_labels, y_pred))
print("Classification Report:")
print(classification_report(test_labels, y_pred))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\incomplete.0RBOG7_1.0.0\imdb_reviews-test.…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\incomplete.0RBOG7_1.0.0\imdb_reviews-unsup…

Dataset imdb_reviews downloaded and prepared to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.
Accuracy: 0.88468
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.88      0.88     12500
           1       0.88      0.89      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



In [5]:
pip install tensorflow_datasets

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   ------- -------------------------------- 1.0/5.3 MB 8.4 MB/s eta 0:00:01
   --------------------- ------------------ 2.9/5.3 MB 8.0 MB/s eta 0:00:01
   ----------------------------------- ---- 4.7/5.3 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 5.3/5.3 MB 8.1 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21544 sha256=b4af0d2d63212f057ca5f7e7928683f5ad6f2ac0659b76ae7a8c2a126a3446dd
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\e7\e6\28\864bdfee5339dbd6ddcb5a186286a8e217648ec198bdf0097d
Successfully built promise
Note: you may need to restart the kernel to use updated packages.


In [13]:
# Import necessary libraries
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords (run this once)
nltk.download('stopwords')

# IMDB dataset from TensorFlow Datasets (train and test)
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

# Split the dataset into train and test
train_data, test_data = dataset['train'], dataset['test']

# Preprocess the text data: remove stopwords, tokenize, and convert to string format
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Check if the input is a TensorFlow tensor (which would need .numpy()) or a plain string
    if isinstance(text, tf.Tensor):
        text_str = text.numpy().decode('utf-8')  # Decode if it's a tensor
    else:
        text_str = text  # If it's a string, use it directly
    
    # Tokenize and remove stopwords
    words = text_str.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

# Convert the train and test data into processed texts and labels
train_texts = [preprocess_text(text) for text, label in train_data]
test_texts = [preprocess_text(text) for text, label in test_data]

train_labels = [label.numpy() for text, label in train_data]
test_labels = [label.numpy() for text, label in test_data]

# Convert texts to TF-IDF features
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_texts)
X_test_tfidf = vectorizer.transform(test_texts)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, train_labels)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(test_labels, y_pred))
print("Classification Report:")
print(classification_report(test_labels, y_pred))

# Example of using the model to predict sentiment on new text
def predict_sentiment(text):
    # Preprocess the text
    text_processed = preprocess_text(text)
    
    # Vectorize the text
    text_tfidf = vectorizer.transform([text_processed])
    
    # Predict sentiment (0: Negative, 1: Positive)
    prediction = model.predict(text_tfidf)
    sentiment = "Positive" if prediction[0] == 1 else "Negative"
    return sentiment

# Test with new sentences
new_text = "I absolutely loved this movie!"
print(f"Sentiment: {predict_sentiment(new_text)}")

new_text = "This movie was terrible, I hated it."
print(f"Sentiment: {predict_sentiment(new_text)}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.88468
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.88      0.88     12500
           1       0.88      0.89      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000

Sentiment: Positive
Sentiment: Negative
